<a href="https://colab.research.google.com/github/yamphurai/Data_Mining_Project1/blob/main/DataMining_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing necessary libraries
library("tidyverse")
library("ggplot2")
library("ggrepel")
library("ggcorrplot")
library("DT")
library(dplyr)
library(tidyr)

install.packages("dplyr")

# Load the dplyr package
library(dplyr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


ERROR: Error in library("ggrepel"): there is no package called ‘ggrepel’


# COVID-19 Global Mobility

In [ ]:
# Retrieve "COVID-19_global_mobility.csv" file
dataset_global_mobility <- read.csv("https://www.dropbox.com/scl/fi/x4i3ah1gmt4kryrrq0c8m/Global_Mobility_Report.csv?rlkey=tt4i5q1zss7e8ly2zmn8xvhc2&st=xnxrxz2d&dl=1")

In [ ]:
names(dataset_global_mobility)

## Structure of the dataset

In [ ]:
str(dataset_global_mobility)

In [ ]:
# Summary of the columns containing "int" data type
summary(dataset_global_mobility[, c("census_fips_code",
                                      "retail_and_recreation_percent_change_from_baseline",
                                      "grocery_and_pharmacy_percent_change_from_baseline",
                                      "parks_percent_change_from_baseline",
                                      "transit_stations_percent_change_from_baseline",
                                      "workplaces_percent_change_from_baseline",
                                      "residential_percent_change_from_baseline")])

In [ ]:
# Count of unique values in these columns
columns_to_check <- c("country_region_code", "country_region", "sub_region_1", "sub_region_2", "metro_area", "iso_3166_2_code", "census_fips_code")
unique_counts <- sapply(dataset_global_mobility[, columns_to_check], function(column) length(unique(column)))
print(unique_counts)

## Missing values

In [ ]:
# Compute the percentage of NA values for each column
na_percentages_global_mobility <- sapply(dataset_global_mobility, function(column) {
  sum(is.na(column)) / nrow(dataset_global_mobility) * 100
})
print(na_percentages_global_mobility)

### Update "date" column with date type values

In [ ]:
# Convert the date column to Date type and extract year and month
dataset_global_mobility <- dataset_global_mobility %>%
  mutate(date = as.Date(date, format = "%Y-%m-%d")) %>%
  mutate(year_month = format(date, "%Y-%m"))

In [ ]:
head(dataset_global_mobility)

In [ ]:
# Drop irrelevant columns
dataset_global_mobility <- dataset_global_mobility %>%
  select(-country_region_code, -metro_area, -sub_region_1, -sub_region_2, -date)

# Remaining feature columns
names(dataset_global_mobility)

In [ ]:
# Checking for missing values in the dataset
na_counts <- colSums(is.na(dataset_global_mobility))

na_summary <- data.frame(
  Column = names((na_counts / 3991405) * 100),
  Missing_Values = round(((na_counts / 3991405) * 100),2),
  stringsAsFactors = FALSE
)

print(na_summary)

In [ ]:
# Statistics of the remaining feature columns
summary(dataset_global_mobility %>% select(retail_and_recreation_percent_change_from_baseline,
                                            grocery_and_pharmacy_percent_change_from_baseline,
                                            parks_percent_change_from_baseline,
                                            transit_stations_percent_change_from_baseline,
                                            workplaces_percent_change_from_baseline,
                                            residential_percent_change_from_baseline))


## Feature processing "grocery_and_pharmacy_percent_change_from_baseline"

In [ ]:
# Replace missing values in retail_and_recreation_percent_change_from_baseline with the corresponding average value for the corresponding
# region within the "iso_3166_2_code" feature column only if the average value is available
dataset_global_mobility <- dataset_global_mobility %>%

    #group the dataset by this column
    group_by(iso_3166_2_code) %>%

    #create updated version of the feature column
    mutate(retail_and_recreation_percent_change_from_baseline =

           # if there is missing value, replace it with mean of that group only if it's available
           ifelse(is.na(retail_and_recreation_percent_change_from_baseline),
                  {
                    avg_value <- mean(retail_and_recreation_percent_change_from_baseline, na.rm = TRUE)
                    if (!is.na(avg_value)) avg_value   #if there is average value
                    else NA
                  },
                  retail_and_recreation_percent_change_from_baseline)) %>%

    # Ungroup after mutation
    ungroup()


In [ ]:
dataset_global_mobility <- dataset_global_mobility %>%
  filter(!is.na(retail_and_recreation_percent_change_from_baseline))

# Checking for missing values
sum(is.na(dataset_global_mobility$retail_and_recreation_percent_change_from_baseline))

## Feature processing "grocery_and_pharmacy_percent_change_from_baseline"

In [ ]:
# Replace missing values in "grocery_and_pharmacy_percent_change_from_baseline" with the corresponding average value for the corresponding
# region within the "iso_3166_2_code" feature column only if the average value is available
dataset_global_mobility <- dataset_global_mobility %>%

    #group the dataset by this column
    group_by(iso_3166_2_code) %>%

    #create updated version of the feature column
    mutate(grocery_and_pharmacy_percent_change_from_baseline =

           # if there is missing value, replace it with mean of that group only if it's available
           ifelse(is.na(grocery_and_pharmacy_percent_change_from_baseline),
                  {
                    avg_value <- mean(grocery_and_pharmacy_percent_change_from_baseline, na.rm = TRUE)
                    if (!is.na(avg_value)) avg_value   #if there is average value
                    else NA
                  },
                  grocery_and_pharmacy_percent_change_from_baseline)) %>%

    # Ungroup after mutation
    ungroup()

In [ ]:
dataset_global_mobility <- dataset_global_mobility %>%
  filter(!is.na(grocery_and_pharmacy_percent_change_from_baseline))

# Checking for missing values
sum(is.na(dataset_global_mobility$grocery_and_pharmacy_percent_change_from_baseline))

## Feature processing "parks_percent_change_from_baseline"

In [ ]:
# Replace missing values in "parks_percent_change_from_baseline" with the corresponding average value for the corresponding
# region within the "iso_3166_2_code" feature column only if the average value is available
dataset_global_mobility <- dataset_global_mobility %>%

    #group the dataset by this column
    group_by(iso_3166_2_code) %>%

    #create updated version of the feature column
    mutate(parks_percent_change_from_baseline =

           # if there is missing value, replace it with mean of that group only if it's available
           ifelse(is.na(parks_percent_change_from_baseline),
                  {
                    avg_value <- mean(parks_percent_change_from_baseline, na.rm = TRUE)
                    if (!is.na(avg_value)) avg_value   #if there is average value
                    else NA
                  },
                  parks_percent_change_from_baseline)) %>%

    # Ungroup after mutation
    ungroup()

dataset_global_mobility <- dataset_global_mobility %>%
  filter(!is.na(parks_percent_change_from_baseline))

# Checking for missing values
sum(is.na(dataset_global_mobility$parks_percent_change_from_baseline))

## Feature processing "transit_stations_percent_change_from_baseline"

In [ ]:
# Replace missing values in "transit_stations_percent_change_from_baseline" with the corresponding average value for the corresponding
# region within the "iso_3166_2_code" feature column only if the average value is available
dataset_global_mobility <- dataset_global_mobility %>%

    #group the dataset by this column
    group_by(iso_3166_2_code) %>%

    #create updated version of the feature column
    mutate(transit_stations_percent_change_from_baseline =

           # if there is missing value, replace it with mean of that group only if it's available
           ifelse(is.na(transit_stations_percent_change_from_baseline),
                  {
                    avg_value <- mean(transit_stations_percent_change_from_baseline, na.rm = TRUE)
                    if (!is.na(avg_value)) avg_value   #if there is average value
                    else NA
                  },
                  transit_stations_percent_change_from_baseline)) %>%

    # Ungroup after mutation
    ungroup()

dataset_global_mobility <- dataset_global_mobility %>%
  filter(!is.na(transit_stations_percent_change_from_baseline))

# Checking for missing values
sum(is.na(dataset_global_mobility$transit_stations_percent_change_from_baseline))

## Feature processing "residential_percent_change_from_baseline"

In [ ]:
# Replace missing values in "residential_percent_change_from_baseline" with the corresponding average value for the corresponding
# region within the "iso_3166_2_code" feature column only if the average value is available
dataset_global_mobility <- dataset_global_mobility %>%

    #group the dataset by this column
    group_by(iso_3166_2_code) %>%

    #create updated version of the feature column
    mutate(residential_percent_change_from_baseline =

           # if there is missing value, replace it with mean of that group only if it's available
           ifelse(is.na(residential_percent_change_from_baseline),
                  {
                    avg_value <- mean(residential_percent_change_from_baseline, na.rm = TRUE)
                    if (!is.na(avg_value)) avg_value   #if there is average value
                    else NA
                  },
                  residential_percent_change_from_baseline)) %>%

    # Ungroup after mutation
    ungroup()

dataset_global_mobility <- dataset_global_mobility %>%
  filter(!is.na(residential_percent_change_from_baseline))

# Checking for missing values
sum(is.na(dataset_global_mobility$residential_percent_change_from_baseline))

In [ ]:
# Remove observations with missing values in "workplaces_percent_change_from_baseline" column
dataset_global_mobility <- dataset_global_mobility %>%
  filter(!is.na(workplaces_percent_change_from_baseline))

In [ ]:
# Checking for missing values
sum(is.na(dataset_global_mobility$workplaces_percent_change_from_baseline))

## Feature processing "census_fips_code"

In [ ]:
# Count NA values in "census_fips_code" for countries other than the United States
na_count_non_us <- dataset_global_mobility %>%
  filter(country_region != "United States") %>%
  summarise(NA_Count = sum(is.na(census_fips_code)))

# Calculate the percentage of NA values
na_percentage_non_us <- (na_count_non_us$NA_Count / 3991405) * 100

# Print the result
print(paste("Percentage of NA values in census_fips_code for countries other than the United States:", round(na_percentage_non_us, 2), "%"))

# Check: Count of NA values for United States in census_fips_code column
na_count_us <- dataset_global_mobility %>%
  filter(country_region == "United States") %>%
  summarise(NA_Count = sum(is.na(census_fips_code)))

print(paste("Count of NA values in census_fips_code for United States:", na_count_us$NA_count))

In [ ]:
# Identify the unique countries other than the United States
unique_countries <- dataset_global_mobility %>%
  filter(country_region != "United States") %>%
  distinct(country_region) %>%
  pull(country_region)

# Create a mapping of these countries to unique dummy numbers
country_dummy_mapping <- data.frame(
  country_region = unique_countries,
  dummy_fips_code = 1:length(unique_countries)
)

# Update the census_fips_code column with these dummy numbers for the corresponding countries
dataset_global_mobility <- dataset_global_mobility %>%
  left_join(country_dummy_mapping, by = "country_region") %>%
  mutate(census_fips_code = ifelse(country_region != "United States" & is.na(census_fips_code),
                                   dummy_fips_code, census_fips_code)) %>%
  select(-dummy_fips_code) # Remove the temporary dummy_fips_code column


In [ ]:
# Print the mapping of countries to dummy values
print(country_dummy_mapping)

Here are the dummy codes assigned to the remaining 134 countries in the "census_fips_code" column.

In [ ]:
# Check for missing values in census_fips_code for the United States
missing_fips_us <- dataset_global_mobility %>%
  filter(country_region == "United States" & is.na(census_fips_code))

# Count the number of missing values
count_missing_fips_us <- nrow(missing_fips_us)

print(paste("Missing census_fips_code values for the United States: ", ((count_missing_fips_us/3991405)*100)))


Since there are only about 0.43% of missing values in the "census_fips_code" feature column for the United States, these missing can be completed removed from the dataset.

In [ ]:
# Remove observations with missing values in "census_fips_code" column for the United States country
dataset_global_mobility <- dataset_global_mobility %>%
  filter(!is.na(census_fips_code))

In [ ]:
sum(is.na(dataset_global_mobility$census_fips_code))

All the missing values in the column "census_fips_code" have been properly handled.

In [ ]:
# Checking for missing values in the dataset after all the feature processing & summary of the updated dataset
sum(is.na(dataset_global_mobility))

In [ ]:
str(dataset_global_mobility)

About 8% of the observations were removed from the original dataset.

## Analysis on COVID-19 impact

### Change in visit to Retail & Recreation

In [ ]:
install.packages('IRkernel')
IRkernel::installspec()


In [ ]:
# Save the updated and cleaned dataset as a CSV file
write.csv(dataset_global_mobility, "dataset_global_mobility_cleaned.csv", row.names = FALSE)

In [ ]:
# Convert year_month to Date type (first day of the month to make it a proper date)
dataset_global_mobility <- dataset_global_mobility %>%
  mutate(year_month = as.Date(paste0(year_month, "-01")))

# Filter for the years 2020 and 2021, extracting the year directly during summarization
filtered_data <- dataset_global_mobility %>%
  filter(format(year_month, "%Y") %in% c("2020", "2021"))

# Calculate the mean percent change for retail and recreation for each country and year
aggregated_data_retail_recreation <- filtered_data %>%
  group_by(country_region, year = format(year_month, "%Y")) %>%
  summarise(mean_percent_change_retail_recreation = mean(retail_and_recreation_percent_change_from_baseline, na.rm = TRUE)) %>%
  ungroup()

# Calculate the overall mean percent change of visit to retail & recreation locations for each country across both years (2020 & 2021)
overall_mean_retail_recreation <- aggregated_data_retail_recreation %>%
  group_by(country_region) %>%
  summarise(overall_mean_change_retail_recreation = mean(mean_percent_change_retail_recreation, na.rm = TRUE)) %>%
  arrange(desc(overall_mean_change_retail_recreation))

# Select only the top 20 countries based on the overall mean percent change
top_countries_retail_recreation <- overall_mean_retail_recreation %>%
  top_n(20, wt = overall_mean_change_retail_recreation)

# Calculate the mean percent change for the United States
us_data <- aggregated_data_retail_recreation %>%
  filter(country_region == "United States")

# Filter the data for top countries and add United States
top_aggregated_data_retail_recreation <- aggregated_data_retail_recreation %>%
  filter(country_region %in% c(top_countries_retail_recreation$country_region, "United States"))

# Plot for top countries including the United States
plot_retail_recreation <- ggplot(top_aggregated_data_retail_recreation, aes(x = reorder(country_region, -mean_percent_change_retail_recreation), y = mean_percent_change_retail_recreation, fill = as.factor(year))) +
  geom_bar(stat = "identity", position = position_dodge()) +
  labs(title = "Top 20 countries with highest % change in visit to retail & recreation in 2020 & 2021 compared to baseline",
       x = "Country",
       y = "Mean Percent Change from Baseline",
       fill = "Year") +
  scale_y_continuous(breaks = seq(-100, 100, by = 5)) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))

print(plot_retail_recreation)

In [ ]:
# Save the plot to a file
ggsave("retail_recreation_plot.png", plot = plot_retail_recreation, width = 12, height = 8, dpi = 300)

### Change in visit to Grocery & Pharmacy

In [ ]:
# Convert year_month to Date type (first day of the month to make it a proper date)
dataset_global_mobility <- dataset_global_mobility %>%
  mutate(year_month = as.Date(paste0(year_month, "-01")))

# Filter for the years 2020 and 2021, extracting the year directly during summarization
filtered_data <- dataset_global_mobility %>%
  filter(format(year_month, "%Y") %in% c("2020", "2021"))

# Calculate the mean percent change for grocery and pharmacy for each country and year
aggregated_data_grocery_pharmacy <- filtered_data %>%
  group_by(country_region, year = format(year_month, "%Y")) %>%
  summarise(mean_percent_change_grocery_pharmacy = mean(grocery_and_pharmacy_percent_change_from_baseline, na.rm = TRUE)) %>%
  ungroup()

# Calculate the overall mean percent change of visit to grocery & pharmacy locations for each country across both years (2020 & 2021)
overall_mean_grocery_pharmacy <- aggregated_data_grocery_pharmacy %>%
  group_by(country_region) %>%
  summarise(overall_mean_change_grocery_pharmacy = mean(mean_percent_change_grocery_pharmacy, na.rm = TRUE)) %>%
  arrange(desc(overall_mean_change_grocery_pharmacy))

# Select only the top 20 countries based on the overall mean percent change
top_countries_grocery_pharmacy <- overall_mean_grocery_pharmacy %>%
  top_n(20, wt = overall_mean_change_grocery_pharmacy)

# Calculate the mean percent change for the United States
usa_data_grocery_pharmacy <- aggregated_data_grocery_pharmacy %>%
  filter(country_region == "United States")

# Filter the data for top countries and add United States
top_aggregated_data_grocery_pharmacy <- aggregated_data_grocery_pharmacy %>%
  filter(country_region %in% c(top_countries_grocery_pharmacy$country_region, "United States"))

# Plot for top countries including the United States
plot_grocery_pharmacy <- ggplot(top_aggregated_data_grocery_pharmacy, aes(x = reorder(country_region, -mean_percent_change_grocery_pharmacy), y = mean_percent_change_grocery_pharmacy, fill = as.factor(year))) +
  geom_bar(stat = "identity", position = position_dodge()) +
  labs(title = "Top 20 countries with highest % change in visit to grocery & pharmacy in 2020 & 2021 compared to baseline",
       x = "Country",
       y = "Mean Percent Change from Baseline",
       fill = "Year") +
  scale_y_continuous(breaks = seq(-100, 100, by = 5)) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))

print(plot_grocery_pharmacy)

# Save the plot
ggsave("plot_grocery_pharmacy_with_usa.png", plot = plot_grocery_pharmacy, width = 10, height = 6)


### Change in visit to Parks

In [ ]:

# Convert year_month to Date type (first day of the month to make it a proper date)
dataset_global_mobility <- dataset_global_mobility %>%
  mutate(year_month = as.Date(paste0(year_month, "-01")))

# Filter for the years 2020 and 2021, extracting the year directly during summarization
filtered_data <- dataset_global_mobility %>%
  filter(format(year_month, "%Y") %in% c("2020", "2021"))

# Calculate the mean percent change for parks for each country and year
aggregated_data_parks <- filtered_data %>%
  group_by(country_region, year = format(year_month, "%Y")) %>%
  summarise(mean_percent_change_parks = mean(parks_percent_change_from_baseline, na.rm = TRUE)) %>%
  ungroup()

# Calculate the overall mean percent change of visit to parks locations for each country across both years (2020 & 2021)
overall_mean_parks <- aggregated_data_parks %>%
  group_by(country_region) %>%
  summarise(overall_mean_change_parks = mean(mean_percent_change_parks, na.rm = TRUE)) %>%
  arrange(desc(overall_mean_change_parks))

# Select only the top 20 countries based on the overall mean percent change
top_countries_parks <- overall_mean_parks %>%
  top_n(20, wt = overall_mean_change_parks)

# Calculate the mean percent change for the United States
us_data_parks <- aggregated_data_parks %>%
  filter(country_region == "United States")

# Filter the data for top countries and add United States
top_aggregated_data_parks <- aggregated_data_parks %>%
  filter(country_region %in% c(top_countries_parks$country_region, "United States"))

# Plot for top countries including the United States
plot_parks <- ggplot(top_aggregated_data_parks, aes(x = reorder(country_region, -mean_percent_change_parks), y = mean_percent_change_parks, fill = as.factor(year))) +
  geom_bar(stat = "identity", position = position_dodge()) +
  labs(title = "Top 20 countries with highest % change in visit to parks in 2020 & 2021 compared to baseline",
       x = "Country",
       y = "Mean Percent Change from Baseline",
       fill = "Year") +
  scale_y_continuous(breaks = seq(-100, 100, by = 5)) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))

print(plot_parks)

# Save the plot
ggsave("plot_parks_with_us.png", plot = plot_parks, width = 10, height = 6)

### Change in visit to Transit Stations

In [ ]:
# Convert year_month to Date type (first day of the month to make it a proper date)
dataset_global_mobility <- dataset_global_mobility %>%
  mutate(year_month = as.Date(paste0(year_month, "-01")))

# Filter for the years 2020 and 2021, extracting the year directly during summarization
filtered_data <- dataset_global_mobility %>%
  filter(format(year_month, "%Y") %in% c("2020", "2021"))

# Calculate the mean percent change for transit stations for each country and year
aggregated_data_transit_stations <- filtered_data %>%
  group_by(country_region, year = format(year_month, "%Y")) %>%
  summarise(mean_percent_change_transit_stations = mean(transit_stations_percent_change_from_baseline, na.rm = TRUE)) %>%
  ungroup()

# Calculate the overall mean percent change of visit to transit stations for each country across both years (2020 & 2021)
overall_mean_transit_stations <- aggregated_data_transit_stations %>%
  group_by(country_region) %>%
  summarise(overall_mean_change_transit_stations = mean(mean_percent_change_transit_stations, na.rm = TRUE)) %>%
  arrange(desc(overall_mean_change_transit_stations))

# Select only the top 20 countries based on the overall mean percent change
top_countries_transit_stations <- overall_mean_transit_stations %>%
  top_n(20, wt = overall_mean_change_transit_stations)

# Calculate the mean percent change for the United States
us_data_transit_stations <- aggregated_data_transit_stations %>%
  filter(country_region == "United States")

# Filter the data for top countries and add United States
top_aggregated_data_transit_stations <- aggregated_data_transit_stations %>%
  filter(country_region %in% c(top_countries_transit_stations$country_region, "United States"))

# Plot for top countries including the United States
plot_transit_stations <- ggplot(top_aggregated_data_transit_stations, aes(x = reorder(country_region, -mean_percent_change_transit_stations), y = mean_percent_change_transit_stations, fill = as.factor(year))) +
  geom_bar(stat = "identity", position = position_dodge()) +
  labs(title = "Top 20 countries with highest % change in visit to transit stations in 2020 & 2021 compared to baseline",
       x = "Country",
       y = "Mean Percent Change from Baseline",
       fill = "Year") +
  scale_y_continuous(breaks = seq(-100, 100, by = 5)) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))

print(plot_transit_stations)

# Save the plot
ggsave("plot_transit_stations_with_us.png", plot = plot_transit_stations, width = 10, height = 6)

### Change in visit to Workplaces

In [ ]:
# Convert year_month to Date type (first day of the month to make it a proper date)
dataset_global_mobility <- dataset_global_mobility %>%
  mutate(year_month = as.Date(paste0(year_month, "-01")))

# Filter for the years 2020 and 2021, extracting the year directly during summarization
filtered_data <- dataset_global_mobility %>%
  filter(format(year_month, "%Y") %in% c("2020", "2021"))

# Calculate the mean percent change for workplaces for each country and year
aggregated_data_workplaces <- filtered_data %>%
  group_by(country_region, year = format(year_month, "%Y")) %>%
  summarise(mean_percent_change_workplaces = mean(workplaces_percent_change_from_baseline, na.rm = TRUE)) %>%
  ungroup()

# Calculate the overall mean percent change of visit to workplaces for each country across both years (2020 & 2021)
overall_mean_workplaces <- aggregated_data_workplaces %>%
  group_by(country_region) %>%
  summarise(overall_mean_change_workplaces = mean(mean_percent_change_workplaces, na.rm = TRUE)) %>%
  arrange(desc(overall_mean_change_workplaces))

# Select only the top 20 countries based on the overall mean percent change
top_countries_workplaces <- overall_mean_workplaces %>%
  top_n(20, wt = overall_mean_change_workplaces)

# Calculate the mean percent change for the United States
us_data_workplaces <- aggregated_data_workplaces %>%
  filter(country_region == "United States")

# Filter the data for top countries and add United States
top_aggregated_data_workplaces <- aggregated_data_workplaces %>%
  filter(country_region %in% c(top_countries_workplaces$country_region, "United States"))

# Plot for top countries including the United States
plot_workplaces <- ggplot(top_aggregated_data_workplaces, aes(x = reorder(country_region, -mean_percent_change_workplaces), y = mean_percent_change_workplaces, fill = as.factor(year))) +
  geom_bar(stat = "identity", position = position_dodge()) +
  labs(title = "Top 20 countries with highest % change in visit to workplaces in 2020 & 2021 compared to baseline",
       x = "Country",
       y = "Mean Percent Change from Baseline",
       fill = "Year") +
  scale_y_continuous(breaks = seq(-100, 100, by = 5)) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))

print(plot_workplaces)

# Save the plot
ggsave("plot_workplaces_with_us.png", plot = plot_workplaces, width = 10, height = 6)


### Change in visit to Residential

In [ ]:
# Convert year_month to Date type (first day of the month to make it a proper date)
dataset_global_mobility <- dataset_global_mobility %>%
  mutate(year_month = as.Date(paste0(year_month, "-01")))

# Filter for the years 2020 and 2021, extracting the year directly during summarization
filtered_data <- dataset_global_mobility %>%
  filter(format(year_month, "%Y") %in% c("2020", "2021"))

# Calculate the mean percent change for residential visits for each country and year
aggregated_data_residential <- filtered_data %>%
  group_by(country_region, year = format(year_month, "%Y")) %>%
  summarise(mean_percent_change_residential = mean(residential_percent_change_from_baseline, na.rm = TRUE)) %>%
  ungroup()

# Calculate the overall mean percent change of visit to residential locations for each country across both years (2020 & 2021)
overall_mean_residential <- aggregated_data_residential %>%
  group_by(country_region) %>%
  summarise(overall_mean_change_residential = mean(mean_percent_change_residential, na.rm = TRUE)) %>%
  arrange(desc(overall_mean_change_residential))

# Select only the top 20 countries based on the overall mean percent change
top_countries_residential <- overall_mean_residential %>%
  top_n(20, wt = overall_mean_change_residential)

# Calculate the mean percent change for the United States
us_data_residential <- aggregated_data_residential %>%
  filter(country_region == "United States")

# Filter the data for top countries and add United States
top_aggregated_data_residential <- aggregated_data_residential %>%
  filter(country_region %in% c(top_countries_residential$country_region, "United States"))

# Plot for top countries including the United States
plot_residential <- ggplot(top_aggregated_data_residential, aes(x = reorder(country_region, -mean_percent_change_residential), y = mean_percent_change_residential, fill = as.factor(year))) +
  geom_bar(stat = "identity", position = position_dodge()) +
  labs(title = "Top 20 countries with highest % change in visit to residential locations in 2020 & 2021 compared to baseline",
       x = "Country",
       y = "Mean Percent Change from Baseline",
       fill = "Year") +
  scale_y_continuous(breaks = seq(-100, 100, by = 5)) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))

print(plot_residential)

# Save the plot
ggsave("plot_residential_with_us.png", plot = plot_residential, width = 10, height = 6)


### Change in visits between 2020 & 2021

In [ ]:
# Convert year_month to Date type (first day of the month to make it a proper date)
dataset_global_mobility <- dataset_global_mobility %>%
  mutate(year_month = as.Date(paste0(year_month, "-01")))

# Filter for the years 2020 and 2021
filtered_data <- dataset_global_mobility %>%
  filter(format(year_month, "%Y") %in% c("2020", "2021"))

# Calculate mean percent changes for visits to public locations
mean_changes <- filtered_data %>%
  group_by(country_region, year = format(year_month, "%Y")) %>%
  summarise(
    mean_retail_recreation = mean(retail_and_recreation_percent_change_from_baseline, na.rm = TRUE),
    mean_grocery_pharmacy = mean(grocery_and_pharmacy_percent_change_from_baseline, na.rm = TRUE),
    mean_parks = mean(parks_percent_change_from_baseline, na.rm = TRUE),
    mean_transit_stations = mean(transit_stations_percent_change_from_baseline, na.rm = TRUE),
    mean_workplaces = mean(workplaces_percent_change_from_baseline, na.rm = TRUE)
  ) %>%
  ungroup()

# Calculate overall mean percent change
overall_mean_changes <- mean_changes %>%
  rowwise() %>%
  mutate(overall_mean_change = mean(c(mean_retail_recreation, mean_grocery_pharmacy,
                                       mean_parks, mean_transit_stations,
                                       mean_workplaces), na.rm = TRUE)) %>%
  ungroup()

# Select only the top 20 countries based on the overall mean percent change
top_countries <- overall_mean_changes %>%
  group_by(country_region) %>%
  summarise(overall_mean = mean(overall_mean_change, na.rm = TRUE)) %>%
  top_n(20, wt = overall_mean) %>%
  pull(country_region)

# Filter the overall mean changes for top countries and include the United States
top_aggregated_mean_changes <- overall_mean_changes %>%
  filter(country_region %in% c(top_countries, "United States"))

# Plot for top countries including the United States
plot_overall_mean_changes <- ggplot(top_aggregated_mean_changes, aes(x = reorder(country_region, -overall_mean_change), y = overall_mean_change, fill = as.factor(year))) +
  geom_bar(stat = "identity", position = position_dodge()) +
  labs(title = "Top 20 countries (including USA) with highest change in visits to public locations between 2020 & 2021",
       x = "Country",
       y = "Average Percent Change from Baseline",
       fill = "Year") +
  scale_y_continuous(breaks = seq(-100, 100, by = 5)) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))

print(plot_overall_mean_changes)

# Save the plot
ggsave("plot_overall_mean_changes.png", plot = plot_overall_mean_changes, width = 10, height = 6)

### Change in visits to public locations in different US states

In [ ]:
# Filter for "United States" from "country_region" column
us_data <- dataset_global_mobility %>%
  filter(country_region == "United States")

# Extract first two digits from 'census_fips_code' to get state codes
us_data <- us_data %>%
  mutate(state_code = substr(census_fips_code, 1, 2))

# Compute mean percent changes for visits to public locations by state
state_avg_change <- us_data %>%
  group_by(state_code) %>%
  summarise(
    mean_retail_recreation = mean(retail_and_recreation_percent_change_from_baseline, na.rm = TRUE),
    mean_grocery_pharmacy = mean(grocery_and_pharmacy_percent_change_from_baseline, na.rm = TRUE),
    mean_parks = mean(parks_percent_change_from_baseline, na.rm = TRUE),
    mean_transit_stations = mean(transit_stations_percent_change_from_baseline, na.rm = TRUE),
    mean_workplaces = mean(workplaces_percent_change_from_baseline, na.rm = TRUE)
  ) %>%
  rowwise() %>%
  mutate(avg_percent_change_US_states = mean(c(mean_retail_recreation, mean_grocery_pharmacy,
                                     mean_parks, mean_transit_stations,
                                     mean_workplaces), na.rm = TRUE)) %>%
  ungroup()




In [ ]:
# Plot the average percentage change for each state
ggplot(state_avg_change, aes(x = state_code, y = avg_percent_change_US_states)) +
  geom_bar(stat = "identity", fill = "skyblue") +
  labs(title = "Average Percentage Change in Visits to Public Locations by State",
       x = "State Code",
       y = "Average Percentage Change") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))

In [2]:
unique(dataset_global_mobility$census_fips_code)

ERROR: Error: object 'dataset_global_mobility' not found
